In [ ]:
import os.path as osp

import torch
from torch.nn import Linear, ReLU, MSELoss

from torch_geometric.nn import SchNet, Sequential
from torch_geometric.loader import DataLoader
from src.data.dataset import Crystals

**Model**

In [ ]:
# TODO: Dimensions for lin layers
model = Sequential(
    'z, pos',
    [
        Schnet(),
        # Linear(),
        # ReLU(inplace=True),
        # Linear()
    ]
).to(device)

**Init**

In [ ]:
save_path = '../data/processed/v1.pt'
dataset = Crystals(save_path)

train_dataset, val_dataset, test_dataset = dataset.get_splits(deterministic=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = MSELoss()

loader = DataLoader(test_dataset, batch_size=64)

**Train loop**

In [ ]:

model.train()
for data in loadeer:
    data = data.to(device)
    pred = model(data.z, data.pos)

    # TODO: make sure pred and data.y have same dim and have corresponding elements
    loss = loss_fn(pred, data.y)
    
    loss.backward()
    optimizer.step()
    print(pred)
    print(pred.shape)
